# LangGraph Concepts: Threads, State, Persistence, and Streaming

This comprehensive tutorial explains core LangGraph concepts through practical examples using AWS Bedrock and Tavily Search.

## Analogy: Think of Your Phone's Messaging App

- **THREAD** = One conversation with a friend (identified by contact name)
- **STATE** = All the messages in that conversation + current context
- **PERSISTENCE** = Your phone saves conversations even when you close the app
- **STREAMING** = Messages appear word-by-word as someone types
- **CHECKPOINT** = Like taking a screenshot of the conversation at any point

## What You'll Learn

1. Creating tools for your agent
2. Understanding and managing STATE
3. Using THREADS for multiple conversations
4. Adding PERSISTENCE to remember conversations
5. STREAMING responses in real-time
6. Human-in-the-loop approval workflows
7. Modifying state and time travel through checkpoints

## Step 1: Installation and Setup

Install all required packages for building LangGraph agents with AWS Bedrock.

In [ ]:
!pip install -q langgraph langchain-aws langchain-core tavily-python

## Step 2: Initial Configuration

### Security Best Practices

Store credentials in Google Colab secrets:
1. Click the 🔑 key icon in the left sidebar
2. Add these secrets:
   - `awsid`: Your AWS Access Key ID
   - `awssecret`: Your AWS Secret Access Key
   - `tavily`: Your Tavily API key from https://tavily.com

### About Tavily

Tavily is the web access layer for AI agents, providing:
- **Fast Search**: Optimized for AI agents with sub-second response times
- **Relevant Results**: Purpose-built for LLMs and RAG applications
- **Real-time Data**: Up-to-date information from across the web
- **Easy Integration**: Simple API with Python, Node.js, and cURL support

Tavily is trusted by 700k+ developers and integrates seamlessly with LangChain.

In [ ]:
import os
import boto3
from langchain_aws import ChatBedrock
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from google.colab import userdata
import json

# Configure AWS credentials
AWS_ACCESS_KEY_ID = userdata.get('awsid')
AWS_SECRET_ACCESS_KEY = userdata.get('awssecret')
AWS_REGION = "us-east-1"

# Configure Tavily API key
os.environ["TAVILY_API_KEY"] = userdata.get('tavily')

# Initialize Bedrock client
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name=AWS_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

# Set up the Bedrock model
llm = ChatBedrock(
    client=bedrock_runtime,
    model_id="amazon.nova-lite-v1:0",
    model_kwargs={
        "temperature": 0.7,
        "max_tokens": 4096
    }
)

print("✓ AWS Bedrock client initialized")
print("✓ Tavily API key configured")
print("\n" + "="*70)
print("READY TO START LEARNING!")
print("="*70)

## Step 3: Create Tools

### What are Tools?

Tools are functions your agent can call to get information or perform actions.
Think of them like apps on your phone - the agent decides when to use them.

We'll create two tools:
1. **Tavily Search**: Search the web for real-time information
2. **Travel Budget Calculator**: Calculate estimated travel costs

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Create Tavily search tool (max 2 results to keep context manageable)
search_tool = TavilySearchResults(max_results=2)

# Create a custom travel budget calculator tool
@tool
def get_travel_budget(destination: str, days: int) -> str:
    """Calculate estimated travel budget for a destination.
    
    Args:
        destination: The city or country to visit
        days: Number of days for the trip
    """
    base_costs = {
        "paris": 200,
        "tokyo": 180,
        "bali": 80,
        "new york": 250,
        "london": 220,
        "default": 150
    }
    
    cost_per_day = base_costs.get(destination.lower(), base_costs["default"])
    total = cost_per_day * days
    
    return f"Estimated budget for {destination} for {days} days: ${total} (${cost_per_day}/day for accommodation + food + local transport)"

tools = [search_tool, get_travel_budget]

print(f"✓ Created {len(tools)} tools")
print(f"  1. {tools[0].name}: {tools[0].description}")
print(f"  2. {tools[1].name}: {tools[1].description}")

## Step 4: Understanding STATE

### What is STATE?

STATE is like the agent's memory. It stores:
- All messages in the conversation
- Current context and information
- What happened so far

### Example

If you ask "Book a hotel in Paris" then ask "What's the weather?",
the STATE remembers you're talking about Paris!

### The `Annotated` Type

The `Annotated[list, add_messages]` part tells LangGraph:
- `list`: Messages are stored in a list
- `add_messages`: New messages get **ADDED** to the list (not replaced)

In [ ]:
# Define our state
class AgentState(TypedDict):
    """The agent's working memory."""
    messages: Annotated[list, add_messages]

print("✓ State defined: Our agent will remember all conversation messages")

## Step 5: Create the Basic Agent (Without Persistence)

### Agent Workflow

Our agent follows this pattern:
1. **Receive input** → Go to agent node
2. **Agent decides**: Do I need tools, or can I answer directly?
   - If tools needed → Go to tools node
   - If no tools needed → END
3. **Tools execute** → Return results to agent
4. **Agent uses tool results** → Formulate final answer → END

This is called a **ReAct loop** (Reasoning and Acting).

In [ ]:
def should_continue(state: AgentState):
    """Decide if we should call tools or finish."""
    messages = state["messages"]
    last_message = messages[-1]
    
    # If the AI wants to use tools, continue to tools
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    # Otherwise, we're done
    return END

def call_model(state: AgentState):
    """Call the AI model."""
    messages = state["messages"]
    response = llm.bind_tools(tools).invoke(messages)
    return {"messages": [response]}

# Create the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("agent", call_model)
workflow.add_node("tools", ToolNode(tools))

# Define the flow
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, ["tools", END])
workflow.add_edge("tools", "agent")

# Compile WITHOUT persistence
basic_agent = workflow.compile()

print("✓ Basic agent created (NO memory between sessions)")

## Step 6: Testing Without Persistence

### The Problem with No Persistence

Watch what happens when the agent has NO memory between calls.
Each `invoke()` is completely independent - the agent forgets everything!

In [ ]:
print("="*70)
print("DEMONSTRATION 1: AGENT WITHOUT PERSISTENCE")
print("="*70)

# First conversation
print("\n👤 User: What's a good budget for 5 days in Tokyo?\n")
response = basic_agent.invoke({
    "messages": [HumanMessage(content="What's a good budget for 5 days in Tokyo?")]
})
print(f"🤖 Agent: {response['messages'][-1].content}")

# Try to reference the previous conversation
print("\n" + "-"*70)
print("👤 User: What about for 7 days?\n")
response = basic_agent.invoke({
    "messages": [HumanMessage(content="What about for 7 days?")]
})
print(f"🤖 Agent: {response['messages'][-1].content}")

print("\n⚠️  PROBLEM: The agent is confused! It doesn't remember Tokyo!")
print("    This is because we didn't save the state between calls.")

## Step 7: Adding Persistence (Checkpointing)

### What is Persistence?

**PERSISTENCE** = Saving the state so conversations can continue later

**CHECKPOINT** = A snapshot of the state at a specific point in time

Think of it like saving a video game - you can come back later and continue where you left off!

### Types of Checkpointers

- **MemorySaver**: Saves in RAM (lost when program ends)
- **SqliteSaver**: Saves to database (persists between runs)

We'll use MemorySaver for this tutorial since it's simpler.

In [ ]:
# Create a checkpointer
memory = MemorySaver()

# Compile WITH persistence
agent_with_memory = workflow.compile(checkpointer=memory)

print("✓ Agent with memory created!")
print("  Now the agent will remember conversations within each thread.")

## Step 8: Understanding Threads

### What is a Thread?

**THREAD** = A unique conversation identified by an ID

### Analogy: WhatsApp Chats

Imagine your WhatsApp app:
- Thread `'alice'` = Your chat with Alice
- Thread `'bob'` = Your chat with Bob
- Thread `'family'` = Your family group chat

### Key Points

- Each thread has its own STATE (conversation history)
- You can have multiple threads running at the same time
- Threads are isolated - they don't interfere with each other

## Step 9: Using Threads - Practical Example

Let's simulate helping TWO different users at the same time.
Each user gets their own thread, maintaining separate conversation contexts.

In [ ]:
print("="*70)
print("DEMONSTRATION 2: USING THREADS")
print("="*70)

# User 1's conversation (Thread: user_sarah)
print("\n👤 USER 1 (Sarah) - Thread ID: 'user_sarah'")
print("-" * 70)

config_sarah = {"configurable": {"thread_id": "user_sarah"}}

response = agent_with_memory.invoke({
    "messages": [HumanMessage(content="I want to plan a trip to Paris for 3 days")]
}, config=config_sarah)

print(f"🤖 Response: {response['messages'][-1].content}\n")

# User 2's conversation (Thread: user_john)
print("👤 USER 2 (John) - Thread ID: 'user_john'")
print("-" * 70)

config_john = {"configurable": {"thread_id": "user_john"}}

response = agent_with_memory.invoke({
    "messages": [HumanMessage(content="I'm thinking of visiting Tokyo for a week")]
}, config=config_john)

print(f"🤖 Response: {response['messages'][-1].content}\n")

# Continue Sarah's conversation
print("👤 USER 1 (Sarah) continues - SAME thread 'user_sarah'")
print("-" * 70)

response = agent_with_memory.invoke({
    "messages": [HumanMessage(content="What about for 5 days instead?")]
}, config=config_sarah)

print(f"🤖 Response: {response['messages'][-1].content}")
print("✓ The agent remembered we were talking about PARIS!\n")

# Continue John's conversation
print("👤 USER 2 (John) continues - SAME thread 'user_john'")
print("-" * 70)

response = agent_with_memory.invoke({
    "messages": [HumanMessage(content="What's the weather like there now?")]
}, config=config_john)

print(f"🤖 Response: {response['messages'][-1].content}")
print("✓ The agent remembered we were talking about TOKYO!\n")

print("🎯 KEY INSIGHT: Each thread maintains its OWN conversation!")
print("   Sarah's thread knows about Paris, John's thread knows about Tokyo")

## Step 10: Inspecting State

You can look inside any thread's state to see:
- All messages in the conversation
- What the next step will be
- The checkpoint configuration

In [ ]:
# Get Sarah's state
sarah_state = agent_with_memory.get_state(config_sarah)

print("👤 Sarah's conversation STATE:")
print("-" * 70)
print(f"Number of messages: {len(sarah_state.values['messages'])}")
print(f"Next step: {sarah_state.next}")
print("\nMessages in order:")

for i, msg in enumerate(sarah_state.values['messages'], 1):
    msg_type = type(msg).__name__
    content = msg.content if hasattr(msg, 'content') else str(msg)
    # Truncate long messages
    if len(str(content)) > 100:
        content = str(content)[:100] + "..."
    print(f"  {i}. {msg_type}: {content}")

## Step 11: Streaming Responses

### What is Streaming?

**STREAMING** = Getting responses in real-time, not waiting for completion

Like seeing '...' when someone is typing on WhatsApp!

### Two Types of Streaming

1. **Stream EVENTS**: See each step (tool call, tool result, final answer)
2. **Stream TOKENS**: See words generated one-by-one (not covered here)

We'll demonstrate event streaming to see the agent's reasoning process.

In [ ]:
print("="*70)
print("DEMONSTRATION 3: STREAMING EVENTS")
print("="*70)

config_streaming = {"configurable": {"thread_id": "user_streaming_demo"}}

print("\n👤 User asks: 'What are the best attractions in Bali?'")
print("="*70)
print("\n🔄 STREAMING EVENTS (step-by-step):\n")

for event in agent_with_memory.stream({
    "messages": [HumanMessage(content="What are the best attractions in Bali?")]
}, config=config_streaming):
    
    for node_name, node_output in event.items():
        print(f"📍 Node: {node_name}")
        
        if "messages" in node_output:
            message = node_output["messages"][-1]
            
            if hasattr(message, 'tool_calls') and message.tool_calls:
                print(f"   🔧 AI wants to use tool: {message.tool_calls[0]['name']}")
                print(f"   📝 Arguments: {message.tool_calls[0]['args']}")
            elif hasattr(message, 'content') and message.content:
                print(f"   💬 Response: {message.content[:100]}...")
            elif hasattr(message, 'name'):
                print(f"   🛠️  Tool '{message.name}' executed")
        
        print()

print("✓ Streaming complete! You saw each step as it happened.")

## Step 12: Human-in-the-Loop (Interrupting for Approval)

### What is Human-in-the-Loop?

**HUMAN-IN-THE-LOOP** = Pausing execution to get human approval before taking actions

### Use Cases

- Approve before booking a flight
- Approve before making a purchase  
- Approve before sending an email
- Review tool calls before execution

### How It Works

We use `interrupt_before=["tools"]` to pause execution BEFORE the agent calls tools.
This lets us review what the agent wants to do and approve or modify it.

In [ ]:
# Create an agent that interrupts before using tools
agent_with_approval = workflow.compile(
    checkpointer=memory,
    interrupt_before=["tools"]  # ⚠️ PAUSE before using tools!
)

print("✓ Agent created with interrupt BEFORE tools")
print("\nLet's try it...")
print("="*70)

config_approval = {"configurable": {"thread_id": "user_approval_demo"}}

print("\n👤 User: 'Search for luxury hotels in Bali'\n")

# This will stop BEFORE calling the search tool
response = agent_with_approval.invoke({
    "messages": [HumanMessage(content="Search for luxury hotels in Bali")]
}, config=config_approval)

print("🤖 Agent wants to take action but is PAUSED!\n")

# Check the state
current_state = agent_with_approval.get_state(config_approval)
print(f"📊 Current state:")
print(f"   Next node: {current_state.next}")
print(f"   Status: Waiting for approval to proceed to '{current_state.next[0]}'\n")

# Look at what the agent wants to do
last_message = current_state.values['messages'][-1]
if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
    tool_call = last_message.tool_calls[0]
    print(f"🔧 Agent wants to call: {tool_call['name']}")
    print(f"📝 With arguments: {json.dumps(tool_call['args'], indent=2)}\n")

# Simulate human approval
print("👨‍💼 Human reviews and APPROVES\n")
print("▶️  Resuming execution...\n")

# Continue execution by passing None (no new input, just continue)
for event in agent_with_approval.stream(None, config=config_approval):
    for node_name, node_output in event.items():
        if node_name == "tools":
            print(f"🛠️  Tool executed!")
        elif node_name == "agent" and "messages" in node_output:
            msg = node_output["messages"][-1]
            if hasattr(msg, 'content') and msg.content:
                print(f"💬 Final response: {msg.content[:150]}...")

print("\n✓ Execution completed after human approval!")

## Step 13: Modifying State (Editing the Past)

### Why Modify State?

You can MODIFY the state to:
- Correct mistakes in the conversation
- Change what the agent is about to do
- Mock tool responses for testing
- Override agent decisions

### Use Case

The agent wants to search for hotels in London, but you want Paris instead.
We can modify the tool call arguments before execution!

In [ ]:
config_modify = {"configurable": {"thread_id": "user_modify_demo"}}

print("="*70)
print("DEMONSTRATION 4: MODIFYING STATE")
print("="*70)

print("\n👤 User: 'What's a good budget for 5 days in London?'\n")

# Pause before tools
response = agent_with_approval.invoke({
    "messages": [HumanMessage(content="What's a good budget for 5 days in London?")]
}, config=config_modify)

# Get the current state
current_state = agent_with_approval.get_state(config_modify)
last_message = current_state.values['messages'][-1]

print("🤖 Agent wants to call tool:")
if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
    print(f"   Tool: {last_message.tool_calls[0]['name']}")
    print(f"   Args: {last_message.tool_calls[0]['args']}\n")

print("👨‍💼 Human: 'Wait! Change it to Paris instead!'\n")

# Modify the tool call
if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
    tool_call_id = last_message.tool_calls[0]['id']
    
    modified_message = AIMessage(
        content="",
        tool_calls=[{
            'name': 'get_travel_budget',
            'args': {'destination': 'Paris', 'days': 5},  # Changed!
            'id': tool_call_id
        }]
    )
    
    # Update the state
    agent_with_approval.update_state(
        config_modify,
        {"messages": [modified_message]}
    )
    
    print("✓ State modified! Changed destination from London to Paris\n")

# Check the new state
new_state = agent_with_approval.get_state(config_modify)
updated_message = new_state.values['messages'][-1]
print(f"🔄 Updated tool call:")
if hasattr(updated_message, 'tool_calls'):
    print(f"   Tool: {updated_message.tool_calls[0]['name']}")
    print(f"   Args: {updated_message.tool_calls[0]['args']}\n")

# Continue execution
print("▶️  Continuing with modified state...\n")
response = agent_with_approval.invoke(None, config=config_modify)
print(f"🤖 Final answer: {response['messages'][-1].content[:200]}...")

## Step 14: Time Travel (Accessing Checkpoint History)

### What is Time Travel?

**CHECKPOINTS** save a snapshot after EACH step in the workflow.

You can go back to ANY previous checkpoint and continue from there!

### Analogy

Like Git commits - you can checkout any previous state and continue development from that point.

### Use Cases

- Debug what went wrong at a specific step
- Test alternative conversation paths
- Recover from mistakes
- Analyze agent decision-making

In [ ]:
config_history = {"configurable": {"thread_id": "user_history_demo"}}

print("="*70)
print("DEMONSTRATION 5: TIME TRAVEL")
print("="*70)

print("\nLet's have a conversation with multiple turns...\n")

messages_to_send = [
    "What's the budget for 3 days in Tokyo?",
    "Now check Paris for 4 days",
    "What about Bali for a week?"
]

for user_msg in messages_to_send:
    print(f"👤 {user_msg}")
    response = agent_with_memory.invoke({
        "messages": [HumanMessage(content=user_msg)]
    }, config=config_history)
    print(f"🤖 {response['messages'][-1].content[:100]}...\n")

# Now let's look at the history
print("="*70)
print("📜 VIEWING CHECKPOINT HISTORY")
print("="*70)

history = list(agent_with_memory.get_state_history(config_history))

print(f"\nTotal checkpoints saved: {len(history)}\n")
print("Checkpoints (most recent first):")

for i, state in enumerate(history[:5]):  # Show first 5
    print(f"\n{i+1}. Checkpoint ID: {state.config['configurable']['checkpoint_id'][:8]}...")
    print(f"   Messages in state: {len(state.values['messages'])}")
    print(f"   Next step: {state.next}")
    if state.values['messages']:
        last_msg = state.values['messages'][-1]
        msg_type = type(last_msg).__name__
        print(f"   Last message type: {msg_type}")

# Let's go back to an earlier checkpoint
print("\n" + "="*70)
print("⏪ TIME TRAVEL: Going back to checkpoint 3")
print("="*70)

if len(history) >= 3:
    old_checkpoint = history[2]  # Third most recent
    
    print(f"\nGoing back to checkpoint with {len(old_checkpoint.values['messages'])} messages")
    print(f"We'll continue the conversation from there...\n")
    
    print("👤 (From the past) Now what about New York for 2 days?")
    
    response = agent_with_memory.invoke({
        "messages": [HumanMessage(content="Now what about New York for 2 days?")]
    }, config=old_checkpoint.config)  # Use the old checkpoint's config!
    
    print(f"\n🤖 {response['messages'][-1].content}")
    print("\n✓ We successfully continued from an earlier point in time!")

## Summary: Key Concepts

### 1. STATE
- The agent's memory at any point in time
- Contains conversation history and context
- Like RAM in a computer

### 2. THREAD
- A unique conversation identified by an ID
- Each thread has its own STATE
- Like separate chat conversations with different people

### 3. PERSISTENCE / CHECKPOINTING
- Saving state snapshots so you can come back later
- Checkpoint = snapshot after each step
- Like saving a video game

### 4. STREAMING
- Getting responses in real-time
- Two types: events (steps) and tokens (words)
- Like seeing typing indicators

### 5. HUMAN-IN-THE-LOOP
- Pausing execution for human approval
- `interrupt_before` = pause before certain nodes
- Use for critical actions that need approval

### 6. STATE MODIFICATION
- You can edit the state at any point
- Change what the agent is about to do
- Correct mistakes or provide mock data

### 7. TIME TRAVEL
- Access any previous checkpoint
- Continue from that point
- Like Git checkout

## Practical Uses

✓ Customer support chatbots (each customer = 1 thread)

✓ Multi-user applications (each user = 1 thread)

✓ Complex workflows requiring approval

✓ Debugging and testing (time travel, mocking)

✓ Long-running tasks that can resume later

## Practice Exercises

Try these exercises to solidify your understanding:

### Exercise 1: Create and Inspect a Thread
Create a new thread with `thread_id="practice1"`. Ask about travel to 3 different cities, then inspect the state to see all messages.

### Exercise 2: Multiple Simultaneous Threads
Create TWO threads running simultaneously. Have different conversations in each and verify they don't interfere with each other.

### Exercise 3: Human-in-the-Loop
Use `interrupt_before` to pause before tool execution. Modify the tool arguments before continuing.

### Exercise 4: Time Travel
Create a conversation with 5+ turns. Go back to the 3rd checkpoint and continue from there with a different question.

In [ ]:
# Exercise 1: Starter Code
my_config = {"configurable": {"thread_id": "practice1"}}

# Your code here:
# 1. Ask about 3 different cities
# 2. Inspect the state

# Example:
# response = agent_with_memory.invoke({
#     "messages": [HumanMessage(content="Your question here")]
# }, config=my_config)

# state = agent_with_memory.get_state(my_config)
# print(f"Messages: {len(state.values['messages'])}")

## Congratulations! 🎉

You now understand:

✓ How threads keep conversations separate

✓ How state stores information

✓ How persistence saves state

✓ How streaming gives real-time feedback

✓ How human-in-the-loop adds control

✓ How to modify state and time travel

### Next Steps

- Explore the [LangGraph documentation](https://langchain-ai.github.io/langgraph/)
- Try building your own agent with custom tools
- Experiment with different checkpointer types (SqliteSaver for persistence)
- Learn about streaming tokens for character-by-character output

Happy coding! 🚀